# Day 18
## Part 1
This seems so simple that I'm nervous part 2.

In [36]:
def parse_data(s):
    return {tuple(int(x) for x in line.split(',')) 
            for line in s.strip().splitlines()}

test_string = """
2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5
"""

test_data = parse_data(test_string)
test_data

{(1, 2, 2),
 (1, 2, 5),
 (2, 1, 2),
 (2, 1, 5),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 2, 4),
 (2, 2, 6),
 (2, 3, 2),
 (2, 3, 5),
 (3, 2, 2),
 (3, 2, 5)}

In [37]:
def part_1(data):
    sides = 0
    for x, y, z in data:
        for d in -1, 1:
            sides += sum([
                (x + d, y, z) not in data, 
                (x, y + d, z) not in data,
                (x, y, z + d) not in data
            ])
    return sides

part_1(test_data)

64

In [38]:
data = parse_data(open("input").read())
part_1(data)

4444

## Part 2

Yep. How am I going to do this? I'll have a think during the football.

Right, vectors. Use a location and vector pair to represent each open face. Put them in a graph. Connect them if they're linked and examine the independent subgraphs. Either do some fancy maths I haven't thought of to work out which is the exterior or, more likely, iterate through the sizes of the subgraphs largest first.

In [68]:
# ... a whole load of stuff that didn't work and took ages to debug

Ok, scrap that. Let's do a search from each face to a point that is definitely outside the shape.

In [70]:
from heapq import heappop, heappush

def manhattan(p1, p2):
    return sum(abs(x1 - x2) for x1, x2 in zip(p1, p2))

def search(p, goal, cubes):
    seen = set((manhattan(p, goal), p))
    q = [(manhattan(p, goal), p)]
    i = 0
    while q:
        m, p = heappop(q)
        if p == goal: 
            return True
        seen.add(p)
        x, y, z = p
        for d in -1, 1:
            nbr = (x + d, y, z)
            if nbr not in seen and nbr not in cubes:
                heappush(q, (manhattan(nbr, goal), nbr))
                seen.add(nbr)
            nbr = (x, y + d, z)
            if nbr not in seen and nbr not in cubes:
                heappush(q, (manhattan(nbr, goal), nbr))
                seen.add(nbr)
            nbr = (x, y, z + d)
            if nbr not in seen and nbr not in cubes:
                heappush(q, (manhattan(nbr, goal), nbr))
                seen.add(nbr)
    return False
    

def part_2(data):
    g = nx.Graph()
    max_x, max_y, max_z = 0, 0, 0
    for x, y, z in data:
        max_x = max(x, max_x)
        max_y = max(y, max_y)
        max_z = max(z, max_z)
        for d in -1, 1:
            if (x + d, y, z) not in data:
                g.add_node(((x, y, z), (d, 0, 0)))
            if (x, y + d, z) not in data:
                g.add_node(((x, y, z), (0, d, 0)))
            if (x, y, z + d) not in data:
                g.add_node(((x, y, z), (0, 0, d)))
                
    goal = (max_x + 1, max_y + 1, max_z + 1)
    
    outer_surfaces = 0
    for p, v in g:
        x, y, z = p
        vx, vy, vz = v
        if search((x + vx, y + vy, z + vz), goal, data):
            outer_surfaces += 1
            
    return outer_surfaces
        
        
part_2(my_data)

54

In [53]:
part_2(test_data)

58

In [54]:
part_2(data)

2530

In [71]:
%%time
part_2(data)

CPU times: user 4.76 s, sys: 6.56 ms, total: 4.76 s
Wall time: 4.76 s


2530